<a href="https://colab.research.google.com/github/Fulea-Andrei/ProjectInfoVisualizationCourse/blob/main/info_visualization_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import re
from collections import Counter
from tqdm import tqdm
from nltk import everygrams
from nltk import FreqDist
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RANSACRegressor
from nltk.stem.snowball import RomanianStemmer
import unidecode
from nltk import sent_tokenize
from sklearn.linear_model import Lasso
import nltk
from nltk import word_tokenize
import os
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
import seaborn as sbn
import pandas as pd
import re
import stop_words
from stop_words import get_stop_words
from nltk.stem.snowball import RomanianStemmer
import unidecode
from nltk.util import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support as score
import missingno as msno

In [ ]:
completed_websites_dataset = pd.read_csv("https://raw.githubusercontent.com/Fulea-Andrei/ProjectInfoVisualizationCourse/main/national_real_estate_data.csv",index_col=0,low_memory=False)
national_real_estate_data = completed_websites_dataset.copy()
ds_cleaned = pd.read_csv("https://raw.githubusercontent.com/Fulea-Andrei/ProjectInfoVisualizationCourse/main/national_real_estate_data_CLEANED.csv",index_col=0)
world_cities = pd.read_csv("https://raw.githubusercontent.com/Fulea-Andrei/ProjectInfoVisualizationCourse/main/worldcities.csv",index_col=0)
ds_labeled = pd.read_csv("https://raw.githubusercontent.com/Fulea-Andrei/DissertationProject/main/date/dataset_labeled.csv?token=GHSAT0AAAAAABUEP42VC5WH72LXOKI24J3WYUGCSEQ",index_col=0)
print(national_real_estate_data.columns)
city_ids = {
    'maramures': 1642377306,
    'iasi' : 1642367695,
    'timis' : 1642603121,
    'timisoara' : 1642603121,
    'brasov' : 1642328429,
    'bucuresti' : 1642414442,
    'sibiu' : 1642393086,
    'cluj' : 1642503974,
    'constanta' : 1642851858,
    'bucuresti-ilfov' :  1642918141,
    'oradea' : 1642066626
}


In [ ]:
class CleaningDataSet():
  def __init__(self, data_frame):
    self.data_frame = data_frame

  def bathrooms_(self):
    bathrooms = list(self.data_frame['bathrooms'].values)
    bathrooms_list_cleaned_int = [int(x) for x in bathrooms] 
    return bathrooms_list_cleaned_int


  def town_mapping(self):
    cities_string = list(self.data_frame['town'].values)
    cities_numbers = []
    for city in tqdm(cities_string):
       cities_numbers.append(city_ids[city])
    return cities_numbers
      

  def get_cosine(self,vec1, vec2):
      intersection = set(vec1.keys()) & set(vec2.keys())
      numerator = sum([vec1[x] * vec2[x] for x in intersection])

      sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
      sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
      denominator = math.sqrt(sum1) * math.sqrt(sum2)

      if not denominator:
          return 0.0
      else:
          return float(numerator) / denominator


  def text_to_vector(self,text):
      try:
        words = self.WORD.findall(text)
      except Exception as err:
        print(err)
      else:
        return Counter(words)
                       

  def labeling_data_set(self):

    tags_luxury_assets = ['spatioasa','mobilata','nou','investitie','bloc nou','centru','utilata','modern','calitate']
    tags_nonluxury_assets = ['margine','nemobilat']

    
    descriptions = list(self.data_frame['description'].values)
    contor = 0
    dataset_labeled = self.data_frame.copy()
    dataset_labeled['label'] = [''] * len(dataset_labeled)

    for desc in tqdm(descriptions):
      if pd.isna(desc) == False:
          for tag in tags_luxury_assets:
            clf = CleaningDataSet(self.data_frame)
            tag_ = clf.text_to_vector(tag,0)
            desc_ = clf.text_to_vector(desc,1)
            cosine = clf.get_cosine(tag_, desc_)
            if cosine > 0.0:
              contor += 1
          
          num_feature = dataset_labeled.loc[dataset_labeled['description'] == desc, 'price'].iloc[0]
   
          if (contor >= len(tags_luxury_assets)/2) and (len(word_tokenize(desc)) > 150): #and (num_feature > 80000):
            dataset_labeled.loc[dataset_labeled['description'] == desc, 'label'] = "luxury_asset"
      elif pd.isna(desc) == True:
            dataset_labeled.loc[dataset_labeled['description'] == desc, 'label'] = "nan"

    ##########################################################################
    for desc in tqdm(descriptions):
        if pd.isna(desc) == False:
            for tag in tags_nonluxury_assets:
              clf = CleaningDataSet(self.data_frame)
              tag_ = clf.text_to_vector(tag,0)
              desc_ = clf.text_to_vector(desc,1)
              cosine = clf.get_cosine(tag_, desc_)
              if cosine > 0.0:
                contor += 1

            if (contor >= len(tags_nonluxury_assets)/2) and (len(word_tokenize(desc)) <= 150): #and (num_feature <= 80000):
              dataset_labeled.loc[dataset_labeled['description'] == desc, 'label'] = "modest_asset"
        elif pd.isna(desc) == True:
              dataset_labeled.loc[dataset_labeled['description'] == desc, 'label'] = "nan"
            


    return dataset_labeled


  def assetstate_map(self):

    assetstate_map = []
    assetstate_list = []
    assetstate_list_map = []
    assetstate_map = list(set(self.data_frame['assetstate'].values))
    assetstate_list = list(self.data_frame['assetstate'].values)
    for i in assetstate_list:
        if i in assetstate_map:
            assetstate_list_map.append(assetstate_map.index(i))
            
    return assetstate_list_map



  def colector_map(self):

    colector_map = []
    colector_list = []
    colector_list_map = []
    colector_map = list(set(self.data_frame['colector'].values))
    colector_list = list(self.data_frame['colector'].values)
    for i in colector_list:
        if i in colector_map:
            colector_list_map.append(colector_map.index(i))
            
    return colector_list_map

  def compartimentation_map(self):

    aux_wrong_list = ['semidecomandat','decomandat','circular','vagon','Nedecomandat','semi-decomandat','comandat','decomandat; det_comp1','nedecomandat']
    aux_right_list = ['Semidecomandat','Decomandat','Circular','Vagon','Comandat','Semidecomandat','Comandat','Decomandat','Comandat']
    compartimentation_map = []
    compartimentation_list = []
    compartimentation_list_map = []
    compartimentation_list = list(self.data_frame['compartimentation'].values)
    
    for i,elem in enumerate(compartimentation_list):
        if elem in aux_wrong_list:
            compartimentation_list[i] = aux_right_list[aux_wrong_list.index(elem)] 
        
    
    compartimentation_map = list(set(compartimentation_list))
    for i in compartimentation_list:
        if i in compartimentation_map:
            compartimentation_list_map.append(compartimentation_map.index(i))
            
    return compartimentation_list_map

  def confort_map(self):

    
    confort_map = []
    confort_list = []
    confort_list_map = []
    confort_list = list(self.data_frame['confort'].values)
    
    for i,elem in enumerate(confort_list):
        if elem == 'lux':
            confort_list[i] = 'Lux' 
        
    confort_map = list(set(confort_list))
    for i in confort_list:
        if i in confort_map:
            confort_list_map.append(confort_map.index(i))
            
    return confort_list_map

  def furnished_map(self):

    
    furnished_map = []
    furnished_list = []
    furnished_list_map = []
    furnished_list = list(self.data_frame['furnished'].values)
    
    for i,elem in enumerate(furnished_list):
        if elem == 'mobilat' or elem == 'mobilat; det_mobila6':
            furnished_list[i] = 'Mobilat' 
               
    furnished_map = list(set(furnished_list))
    for i in furnished_list:
        if i in furnished_map:
            furnished_list_map.append(furnished_map.index(i))
            
    return furnished_list_map


  def rooms_cleaning(self):

    rooms_list_cleaned = []
    rooms_list = list(self.data_frame['rooms'].values)
    for i,elem in enumerate(rooms_list):
        if type(elem) is float:
            rooms_list_cleaned.append("0")
        elif len(elem) == 1 or len(elem) == 2: 
            rooms_list_cleaned.append(elem)
        elif len(elem) == 3 or len(elem) == 4: 
            lst_split = elem.split(".")
            rooms_list_cleaned.append(lst_split[0])
        elif len(elem) > 4 and len(elem) < 10:
            lst_split = elem.split()
            rooms_list_cleaned.append(lst_split[0])
        elif len(elem) > 9:
            lst_split = elem.split()
            rooms_list_cleaned.append(lst_split[-1])
            
    rooms_list_cleaned_int = [int(x) for x in rooms_list_cleaned]         
    return rooms_list_cleaned_int 

  def balcony_cleaning(self):
    balcony_list_cleaned = []
    balcony_list = list(self.data_frame['balcony'].values)
    for elem in balcony_list:
        if type(elem) is float:
            balcony_list_cleaned.append("0")
        else:
            balcony_list_cleaned.append(elem[0])
    
    balcony_list_cleaned_int = [int(x) for x in balcony_list_cleaned] 
    return balcony_list_cleaned_int

  def squaremetres_cleaning(self):
    squaremetres_list_cleaned = []
    squaremetres_list = list(self.data_frame['squaremetres'].values)
    for elem in squaremetres_list:
        if type(elem) == float:                                        
            squaremetres_list_cleaned.append("0")
        elif type(elem) == str:
            if "," in elem and "." not in elem and " " in elem:   #98,2 mp
                elem = elem.replace(",", ".")
                lst_split = elem.split()
                squaremetres_list_cleaned.append(lst_split[0])
            elif "," in elem and "." not in elem and " " not in elem:   #98,2
                elem = elem.replace(",",".")
                squaremetres_list_cleaned.append(elem)    
            elif "," not in elem and "." in elem and " " in elem:    #98.2 mp      
                lst_split = elem.split()
                squaremetres_list_cleaned.append(lst_split[0])
            elif "," not in elem and "." in elem and " " not in elem:   #98.2
                squaremetres_list_cleaned.append(elem)    
            elif "," not in elem and "." not in elem and " " in elem:  #98 mp    
                lst_split = elem.split()
                squaremetres_list_cleaned.append(lst_split[0])    
        
    squaremetres_list_cleaned_float =  [int(float(x)) for x in squaremetres_list_cleaned] 
    return squaremetres_list_cleaned_float

  def yearconstruction_cleaning(self):
    yearconstruction_list_cleaned = []
    yearconstruction_list_cleaned_int = []
    test = []
    yearconstruction_list = list(self.data_frame['yearconstruction'].values)
    for elem in yearconstruction_list: 
        if type(elem) == float:   #nan                                     
            yearconstruction_list_cleaned.append("0")
        elif type(elem) == str:
            if "." not in elem and len(elem) == 4:   #1998
                yearconstruction_list_cleaned.append(elem)
            elif "." in elem and " " not in elem:   #1998.0
                lst_split = elem.split('.')
                yearconstruction_list_cleaned.append(lst_split[0])    
            elif "-" in elem and " " not in elem:    #1990-1998      
                lst_split = elem.split('-')
                yearconstruction_list_cleaned.append(lst_split[1])
            elif "(" in elem and ")" in elem and " " in elem:   #1998 (in constructie)
                lst_split = elem.split()
                yearconstruction_list_cleaned.append(lst_split[0])    
            elif "şi" in elem and "(" not in elem and ")" not in elem:  #intre 1998 si 1998    
                lst_split = elem.split()
                yearconstruction_list_cleaned.append(lst_split[-1])
            elif "de" in elem and "(" not in elem and ")" not in elem:  #inainte de 1998    
                lst_split = elem.split()
                yearconstruction_list_cleaned.append(lst_split[-1]) 
            else:
                test.append(elem)
    #print(test)    
    yearconstruction_list_cleaned_int =  [int(x) for x in yearconstruction_list_cleaned] 
    return yearconstruction_list_cleaned_int

  def price_cleaning(self):
    price_list = []
    price_list_cleaned = []
    price_list_cleaned_int = []
    ramase = []
    price_list = list(self.data_frame['price'].values)
    
    listuta=[]
    listuta = self.squaremetres_cleaning()
    
    for i,elem in enumerate(price_list):
        if type(elem) == float:
            price_list_cleaned.append("0")
        elif type(elem) == str:         
            if ("€" in elem or "EUR" in elem or "€;" in elem) and ("RON" not in elem and "€/mp" not in elem):    
                list_split = elem.split()
                price_list_cleaned.append(list_split[0].replace(".",""))
                continue
            elif "," in elem and "EUR" in elem:
                price_list_cleaned.append("0")
            elif elem == "bar_chart":
                price_list_cleaned.append("0")
                continue
            elif elem.replace(".", "").isnumeric():
                price_list_cleaned.append(elem.replace(".",""))
                continue
            elif "LEI" in elem:
                list_split = elem.split()
                price_list_cleaned.append(list_split[0].replace(".",""))
                continue
            elif "chart" in elem and "'\'" in elem:
                list_split = elem.split()
                price_list_cleaned.append(list_split[0].replace(".","")) 
                continue
            elif "RON" in elem and "USD" in elem and "EUR" in elem:
                aux1 = elem.split("D")
                aux2 = aux1[1].split()
                price_list_cleaned.append(aux2[0].replace(".",""))
                continue
            elif "RON" in elem and "USD" in elem and "EUR" not in elem: 
                aux1 = elem.split("N")
                aux2 = aux1[1].split()
                aux3 = aux2[0].replace(".", "")
                aux3 = int(aux3)
                aux3 = aux3 * .9
                aux3 = round(aux3)
                price_list_cleaned.append(str(aux3).replace(".","")) 
                continue
            elif "ron" in elem:
                price_list_cleaned.append("0")
                continue
            else:
                list_split = elem.split()
                metri = str(round(listuta[i]))
                pret = int(list_split[0].replace(".","")) * int(metri.replace(".",""))
                price_list_cleaned.append(str(pret).replace(".",""))
    
    price_list_cleaned_int =  [int("0") if "," in x else int(x) for x in price_list_cleaned]            
    return price_list_cleaned_int


datasetCleanerObject = CleaningDataSet(national_real_estate_data)

city_ids = datasetCleanerObject.town_mapping()

assetStateMap = datasetCleanerObject.assetstate_map()
colectorMap = datasetCleanerObject.colector_map()
compartimentationMap = datasetCleanerObject.compartimentation_map()
confortMap = datasetCleanerObject.confort_map()
furnishedMap = datasetCleanerObject.furnished_map()
roomsCleaning = datasetCleanerObject.rooms_cleaning()
balconyCleaning = datasetCleanerObject.balcony_cleaning()
smCleaning = datasetCleanerObject.squaremetres_cleaning()
ycCleaning = datasetCleanerObject.yearconstruction_cleaning()
priceCleaning = datasetCleanerObject.price_cleaning()
# bathrooms_= datasetCleanerObject.bathrooms_()

national_real_estate_data_CLEANED = national_real_estate_data.copy()

national_real_estate_data_CLEANED['assetstate'] = assetStateMap
national_real_estate_data_CLEANED['colector'] = colectorMap
national_real_estate_data_CLEANED['compartimentation'] = compartimentationMap
national_real_estate_data_CLEANED['confort'] = confortMap
national_real_estate_data_CLEANED['furnished'] = furnishedMap
national_real_estate_data_CLEANED['rooms'] = roomsCleaning
national_real_estate_data_CLEANED['balcony'] = balconyCleaning
national_real_estate_data_CLEANED['squaremetres'] = smCleaning
national_real_estate_data_CLEANED['yearconstruction'] = ycCleaning
national_real_estate_data_CLEANED['price'] = priceCleaning
national_real_estate_data_CLEANED['townID'] = city_ids
# national_real_estate_data_CLEANED['bathrooms'] = bathrooms_



new_tp = []
for tp in list(national_real_estate_data_CLEANED['transactiontype']):
  if tp == "vanzari":
    new_tp.append("vanzare")

  elif tp == "inchiriere":
    new_tp.append("inchiriere")

  elif tp == "vanzare":
    new_tp.append("vanzare")
  


national_real_estate_data_CLEANED['transactiontype'] = new_tp


100%|██████████| 45015/45015 [00:00<00:00, 900356.67it/s]


graphics class service

In [ ]:
class Graphics():
  def __init__(self,national_real_estate_data,national_real_estate_data_CLEANED):
    self.national_real_estate_data = national_real_estate_data
    self.national_real_estate_data_CLEANED = national_real_estate_data_CLEANED

  def page_for_each_collector(self):
   
    ds = {}
    # create data
    colectors = list(set(self.national_real_estate_data['colector'].values))
    for colector in colectors:
      page = national_real_estate_data.loc[national_real_estate_data['colector'] == colector, 'pagenumber']
      max_page = max(list(page.values))
      ds[colector] = max_page
     
    keys = ds.keys()
    values = ds.values()
    x = np.arange(len(keys)) 
    width = 0.4
    fig, ax = plt.subplots()
    ax.set_ylabel('')
    ax.set_title('')
    ax.set_xticks(x)
    ax.set_xticklabels(keys)
    pps = ax.bar(x - width / 2, values, width, label='pret')
    for p in pps:
      height = p.get_height()
      ax.text(x=p.get_x() + p.get_width() / 2, y=height+.10,s="{}".format(height),ha='center')
    plt.show()


  def missing_values(self):
    msno.heatmap(self.national_real_estate_data_CLEANED)

  def class_distribution(self):
    class_counts = ds_labeled.groupby('label').size()
    perc1 = str(round((int(class_counts['luxury_asset']) / len(ds_labeled)) * 100,2)) + '%'
    perc2 = str(round((int(class_counts['modest_asset']) / len(ds_labeled)) * 100,2)) + '%'
    perc1, perc2
    class_distributions = pd.DataFrame(data=[[perc1,perc2]],columns=['luxury asset distribution','modest asset distribution'])

    sns.histplot(data=self.national_real_estate_data,x='label')
    plt.show()

  def corr_matrix(self):
    corrMatrix = self.national_real_estate_data_CLEANED[['townID','rooms','yearconstruction','confort','furnished','squaremetres','compartimentation']].corr()
    sns.heatmap(corrMatrix, annot=True)
    plt.show()

  # de adaugat : violin plot, boxplot, pairgrid, jointplot, swarmplot
  def joint_plot(self):
    sns.jointplot(data=self.national_real_estate_data_CLEANED, x="bill_length_mm", y="bill_depth_mm")

  def regression_plot(self):
    plt.figure(figsize=(7,7))
    sns.regplot(x=self.national_real_estate_data_CLEANED["price"] , y = self.national_real_estate_data_CLEANED["rooms"])
    plt.show()


  def swarmplot(self):
    plt.figure(figsize=(10,10))
    sns.swarmplot(self.national_real_estate_data_CLEANED['price'], self.national_real_estate_data_CLEANED['yearconstruction'])
    plt.show()

  def violinplot(self):
    sns.violinplot(x="price", y="rooms",
                    data=self.national_real_estate_data_CLEANED)
    
  def pairgrid(self):
    g = sns.PairGrid(self.national_real_estate_data_CLEANED, hue="townID")
    g.map_upper(sns.scatterplot)
    g.map_lower(sns.kdeplot)
    g.map_diag(sns.kdeplot)

  def pieplot(self):
    size_of_groups=[]
    # create data
    colectors = set(list(self.national_real_estate_data['colector'].values))
    for colector in colectors:
      size_of_groups.append(len(self.national_real_estate_data[self.national_real_estate_data['colector'] == colector]))
    explode = [0.2] * len(size_of_groups)
    plt.pie(size_of_groups, labels = colectors,  autopct='%.1f%%',  explode=explode, shadow=True)
    plt.show()

  def histograms(self):
    fig, axs = plt.subplots(3, 2, figsize=(7, 7))
    sns.histplot(data=self.national_real_estate_data,x='colector',color="skyblue", ax=axs[0, 0])
    sns.histplot(data=self.national_real_estate_data,x='transactiontype', color="olive", ax=axs[0, 1])

    sns.histplot(data=self.national_real_estate_data,y='town', color="gold", ax=axs[1, 0])
    sns.histplot(data=self.national_real_estate_data_CLEANED,x='rooms',  color="black", ax=axs[1, 1])

    sns.histplot(data=self.national_real_estate_data_CLEANED,x='confort', color="green", ax=axs[2, 0])#.set_title('0 reprezinta cel mai inalt grad confort')
    sns.histplot(data=self.national_real_estate_data_CLEANED,x='balcony', color="red", ax=axs[2, 1])#.set_title('0 reprezinta ca nu a fost specificat, 1 exista, 2 nu exista')

    fig.tight_layout()
    plt.show()

  
  def medium_price(self, interest_area_list, title, ytitle, opdecider):
    mean_prices_per_area = {}
    if opdecider == 0:
      for area_ in interest_area_list:
        price_specific_area = self.national_real_estate_data_CLEANED.query("'{}' in area".format(area_))
        list_prices = list(price_specific_area['price'].values)
        if len(list_prices) > 0 :
          mean_prices_per_area[area_] = int(mean(list_prices))

    elif opdecider == 1:
      for area_ in interest_area_list:
        price_specific_area = self.national_real_estate_data_CLEANED.query("yearconstruction == {}".format(area_))
        list_prices = list(price_specific_area['price'].values)
        mean_prices_per_area[area_] = int(mean(list_prices))
    
    elif opdecider == 2:
      for area_ in interest_area_list:
        price_specific_area = self.national_real_estate_data_CLEANED.query("compartimentation == {}".format(area_))
        list_prices = list(price_specific_area['price'].values)
        mean_prices_per_area[area_] = int(mean(list_prices))

    mean_prices_per_area_sorted = dict(sorted(mean_prices_per_area.items(), key=lambda x: x[1], reverse=True))
    df_prices = pd.DataFrame.from_dict([mean_prices_per_area_sorted])
    plt.rcParams["figure.figsize"] = [8, 8]
    plt.rcParams["figudre.autolayout"] = True
    keys = mean_prices_per_area_sorted.keys()
    values = mean_prices_per_area_sorted.values()
    x = np.arange(len(keys)) 
    width = 0.4
    fig, ax = plt.subplots()
    ax.set_ylabel(ytitle)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(keys)
    pps = ax.bar(x - width / 2, values, width, label='pret')
    for p in pps:
      height = p.get_height()
      ax.text(x=p.get_x() + p.get_width() / 2, y=height+.10,s="{}".format(height),ha='center')
    plt.show()


gph = Graphics(national_real_estate_data, national_real_estate_data_CLEANED)
gph.violinplot()


classification problem - asset classification

In [ ]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

class ClassificationProblem():
   def __init__(self, X_train,X_test, y_train, y_test,n_class):
      self.X_train = X_train
      self.X_test = X_test
      self.y_train = y_train
      self.y_test = y_test
      self.n_class = n_class
      
   def knn_classifier(self, n_neighbors):
      model = KNeighborsClassifier(n_neighbors = n_neighbors)
      model.fit(self.X_train, self.y_train)
      y_pred_test = model.predict(self.X_test)
      rep  = classification_report(self.y_test, y_pred_test)
      print(rep)

   def mlp(self):
        mlp = MLPClassifier(
            hidden_layer_sizes=(50, 150, 200),
            activation='relu',
            solver='adam',
            learning_rate='constant',
            early_stopping=True,
            validation_fraction=0.2
        )

        mlp.fit(self.X_train, self.y_train)
        predicted_test = mlp.predict(self.X_test)
        predicted_train = mlp.predict(self.X_train)
        rep  = classification_report(self.y_test, predicted_test)
        print(rep)

      
   def meth_randomforest(self):
        rf_reg = RandomForestClassifier(n_estimators = 750)
        rf_reg.fit(self.X_train, self.y_train)
        test_pred = rf_reg.predict(self.X_test)
        train_pred = rf_reg.predict(self.X_train)
        clf_rep  = classification_report(self.y_test, test_pred)
        print(clf_rep)


   
   def ann(self, bsize, epochs_nr):
        X_train = np.array(self.X_train)
        X_test = np.array(self.X_test)
        y_train = np.array(self.y_train)
        y_test = np.array(self.y_test)

        model = Sequential()
        model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))

        model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.3))

        model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.4))

        model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.5))

        model.add(Dense(self.n_class, activation='softmax'))

        opt = Adam(learning_rate=1e-4)

        model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            batch_size=bsize,
                            epochs=epochs_nr)
        # validation_split=0.2)

        print(model.summary())

        # test_pred = model.predict(self.X_test)
        # train_pred = model.predict(self.X_train)

        try:
            plt.plot(history.history['loss'], label='loss')
            plt.plot(history.history['val_loss'], label='val_loss')

            plt.plot(history.history['val_accuracy'], label='val_accuracy')
            plt.plot(history.history['accuracy'], label='accuracy')

            plt.legend(['loss', 'val_loss', 'val acc', 'acc'])
            plt.show()

        except Exception as err:
            print(err)

   



In [ ]:
ds_labeled['balcony'] = ds_labeled['balcony'].fillna(method='ffill').fillna(method='bfill')
ds_labeled['parkingplaces'] = ds_labeled['parkingplaces'].fillna(method='ffill').fillna(method='bfill')
ds_labeled['bathrooms'] = ds_labeled['bathrooms'].fillna(method='ffill').fillna(method='bfill')
ds_labeled['townID'] = city_ids

label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(ds_labeled['label'])

x = ds_labeled[['townID','rooms','yearconstruction','confort','furnished','squaremetres','compartimentation','balcony','parkingplaces','bathrooms']]

n_class = len(y)
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3)

scaler = StandardScaler()
x_train_standard = scaler.fit_transform(x_train)
x_test_standard = scaler.transform(x_test)
 
mlCommander = ClassificationProblem(x_train_standard, x_test_standard, y_train, y_test, n_class)
# mlCommander.ann(128,30)
# mlCommander.mlp()
print("#"*30)
# mlCommander.knn_classifier(4)
print("#"*30)
mlCommander.meth_randomforest()





regression problem - price prediction





In [30]:
x = national_real_estate_data_CLEANED[['townID','rooms','yearconstruction','confort','furnished','squaremetres','compartimentation','balcony','parkingplaces','bathrooms']]

def cross_val(model):
    pred = cross_val_score(model, x, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted) * 100
    return mae, mse, rmse, r2_square

class RegressionProblem():

  def __init__(self, X_train,X_test, y_train, y_test):
    self.X_train = X_train
    self.X_test = X_test
    self.y_train = y_train
    self.y_test = y_test

  results_df = pd.DataFrame(columns = ['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"] )

  def meth_linear_regr(self):
    lin_reg = LinearRegression(normalize=True)
    lin_reg.fit(self.X_train,y_train)
    coeff_df = pd.DataFrame(lin_reg.coef_, X.columns, columns=['Coefficient'])
    test_pred = lin_reg.predict(self.X_test)
    train_pred = lin_reg.predict(self.X_train)
    # print('Test set evaluation:\n_____________________________________')
    # print_evaluate(y_test, test_pred)
    # print('Train set evaluation:\n_____________________________________')
    # print_evaluate(y_train, train_pred)

    results_df_2 = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                              columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)
    

  def meth_ransac(self):
    model = RANSACRegressor(base_estimator=LinearRegression(), max_trials=100)
    model.fit(self.X_train, self.y_train)

    test_pred = model.predict(self.X_test)
    train_pred = model.predict(self.X_train)

    # print('Test set evaluation:\n_____________________________________')
    # print_evaluate(y_test, test_pred)
    # print('====================================')
    # print('Train set evaluation:\n_____________________________________')
    # print_evaluate(y_train, train_pred)

    results_df_2 = pd.DataFrame(data=[["Robust Regression", *evaluate(y_test, test_pred) , cross_val(RANSACRegressor())]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)

  def meth_ridge(self):
    model = Ridge(alpha=100, solver='cholesky', tol=0.0001, random_state=42)
    model.fit(self.X_train, self.y_train)
    pred = model.predict(self.X_test)

    test_pred = model.predict(self.X_test)
    train_pred = model.predict(self.X_train)

    # print('Test set evaluation:\n_____________________________________')
    # print_evaluate(y_test, test_pred)
    # print('====================================')
    # print('Train set evaluation:\n_____________________________________')
    # print_evaluate(y_train, train_pred)

    results_df_2 = pd.DataFrame(data=[["Ridge Regression", *evaluate(y_test, test_pred) , cross_val(Ridge())]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)

  def meth_lasso(self):
    model = Lasso(alpha=0.1, 
              precompute=True, 
#               warm_start=True, 
              positive=True, 
              selection='random',
              random_state=42)
    model.fit(self.X_train, self.y_train)

    test_pred = model.predict(self.X_test)
    train_pred = model.predict(self.X_train)

    # print('Test set evaluation:\n_____________________________________')
    # print_evaluate(y_test, test_pred)
    # print('====================================')
    # print('Train set evaluation:\n_____________________________________')
    # print_evaluate(y_train, train_pred)

    results_df_2 = pd.DataFrame(data=[["Lasso Regression", *evaluate(y_test, test_pred) , cross_val(Lasso())]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)

  def meth_elasticnet(self):
    model = ElasticNet(alpha=0.1, l1_ratio=0.9, selection='random', random_state=42)
    model.fit(self.X_train, self.y_train)

    test_pred = model.predict(self.X_test)
    train_pred = model.predict(self.X_train)

    # print('Test set evaluation:\n_____________________________________')
    # print_evaluate(y_test, test_pred)
    # print('====================================')
    # print('Train set evaluation:\n_____________________________________')
    # print_evaluate(y_train, train_pred)

    results_df_2 = pd.DataFrame(data=[["Elastic Net Regression", *evaluate(y_test, test_pred) , cross_val(ElasticNet())]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)

  def meth_polynomial(self):
    poly_reg = PolynomialFeatures(degree=2)

    X_train_2_d = poly_reg.fit_transform(self.X_train)
    X_test_2_d = poly_reg.transform(self.X_test)

    lin_reg = LinearRegression(normalize=True)
    lin_reg.fit(X_train_2_d,self.y_train)

    test_pred = lin_reg.predict(X_test_2_d)
    train_pred = lin_reg.predict(X_train_2_d)


    results_df_2 = pd.DataFrame(data=[["Polynomail Regression", *evaluate(y_test, test_pred), 0]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)
  
  def ann(self, epochs_nr,bsize):
      X_train = np.array(self.X_train)
      X_test = np.array(self.X_test)
      y_train = np.array(self.y_train)
      y_test = np.array(self.y_test)

      model = Sequential()

      model.add(Dense(X_train.shape[1], activation='relu'))
      model.add(Dense(64, activation='relu'))
      model.add(Dropout(0.3))

      model.add(Dense(128, activation='relu'))
      model.add(Dropout(0.3))

      model.add(Dense(256, activation='relu'))
      model.add(Dropout(0.4))

      model.add(Dense(512, activation='relu'))
      model.add(Dropout(0.5))

      model.add(Dense(1024, activation='relu'))
      model.add(Dropout(0.5))

      # model.add(Dense(2048, activation='relu'))
      # model.add(Dropout(0.5))

      model.add(Dense(512, activation='relu'))
      model.add(Dropout(0.5))

      model.add(Dense(256, activation='relu'))
      model.add(Dropout(0.5))

      # model.add(Dense(128, activation='relu'))
      # model.add(Dropout(0.4))

      # model.add(Dense(64, activation='relu'))
      # model.add(Dropout(0.2))

      model.add(Dense(1))

      model.compile(optimizer='adam', loss='huber')


      history = model.fit(X_train, y_train,
                    validation_data=(X_test,y_test),
                    batch_size=bsize,
                    epochs=epochs_nr)
                    #validation_split=0.2)
      
      print(model.summary())

      test_pred = model.predict(X_test)
      train_pred = model.predict(X_train)

      # print('Test set evaluation:\n_____________________________________')
      # print_evaluate(y_test, test_pred)

      # print('Train set evaluation:\n_____________________________________')
      # print_evaluate(y_train, train_pred)

      results_df_2 = pd.DataFrame(data=[["Artficial Neural Network", *evaluate(y_train, train_pred), 0]], 
                                  columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
      self.results_df = self.results_df.append(results_df_2, ignore_index=True)


      try:
          plt.plot(history.history['val_loss'], label='val loss')
          plt.plot(history.history['loss'], label='loss')

          plt.legend(['val loss','loss'])
          plt.show()
      
      except Exception as err:
          print(err)



  
  def meth_randomforest(self):
    # from sklearn.model_selection import GridSearchCV
    # param_grid = {
    # 'bootstrap': [True],
    # 'max_depth': [80, 90, 100, 110],
    # 'max_features': [2, 3],
    # 'min_samples_leaf': [3, 4, 5],
    # 'min_samples_split': [8, 10, 12],
    # 'n_estimators': [100, 200, 300, 1000]
    # }
    # # Create a based model
    # rf = RandomForestRegressor()
    # # Instantiate the grid search model
    # grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
    #                           cv = 2, n_jobs = -1, verbose = 2)
    # grid_search.fit(self.X_train, self.y_train)
    # print(grid_search.best_params_)

    rf_reg = RandomForestRegressor(bootstrap = True, max_depth = 110, max_features = 3, min_samples_leaf = 3, min_samples_split =  8, n_estimators = 1500)
    rf_reg.fit(self.X_train, self.y_train)

    test_pred = rf_reg.predict(self.X_test)
    train_pred = rf_reg.predict(self.X_train)


    results_df_2 = pd.DataFrame(data=[["Random Forest Regressor", *evaluate(y_test, test_pred), 0]], 
                                columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
    self.results_df = self.results_df.append(results_df_2, ignore_index=True)




running regression problem

In [ ]:
filling_method = "bfill"

ds_labeled['balcony'] = ds_labeled['balcony'].fillna(method=filling_method)
ds_labeled['parkingplaces'] = ds_labeled['parkingplaces'].fillna(method=filling_method)
ds_labeled['bathrooms'] = ds_labeled['bathrooms'].fillna(method=filling_method)
ds_labeled['townID'] = city_ids
y = ds_labeled['price']

x = ds_labeled[['townID','rooms','yearconstruction','confort','furnished','squaremetres','compartimentation','balcony','parkingplaces','bathrooms']]


x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.33)

scaler = StandardScaler()
x_train_standard = scaler.fit_transform(x_train)
x_test_standard = scaler.transform(x_test)


mlCommander = RegressionProblem(x_train_standard, x_test_standard, y_train, y_test)

# mlCommander.meth_elasticnet()
print("working with ".format(filling_method))
mlCommander.ann(100, 128)
# mlCommander.meth_polynomial()

# mlCommander.meth_randomforest()
# mlCommander.meth_linear_regr()
# mlCommander.meth_ransac()
# mlCommander.meth_ridge()

model_scores = mlCommander.results_df
print(model_scores.to_string())
